# Projet DataScience - Entreprise TouNum

## Méthodologie CRISP :

|Auteurs|Centre|Modification|Date|
|---|---|---|---|
|Alexis Billequin - Clément Chabrier - Etienne Duverney - Adrien Kegler - Nicolas Mazard - Baptiste Saclier - Adrien Thevenet|Lyon|première version|14/01/2020|

### Business Understanding :

#### Demande métier :

Nous travaillons pour TouNum, une entreprise de numérisation des cassettes vidéos.Le but de ce projet est d'améliorer les images de cassette vidéo VHS pour qu'elles soient analysables.

TouNum nous demande donc d'en un premier temps de travailler sur la restauration d'images [`PAL (Phase Alternating Line)`](https://www.wikiwand.com/fr/Phase_Alternating_Line) issu de cassettes VHS grâce au machine learning; Il faudra:
- Détecter/réduire/faire disparaitre les défauts
- Rendre le processus automatique
- Pouvoir s'adapter à tous les types de données d'images
- Rendre la solution lisible sous la forme de notebooks [`Jupyter`](https://jupyter.org/)
- Exporter le modèle de machine learning via [`Pickle`](https://docs.python.org/fr/3.7/library/pickle.html)
- En bonus:
    - Présenter des pistes de réflexions sur l'amélioration du modèle pour le traitement de vidéos

#### Evaluation du résultat :

Pour évaluer l'efficacité de notre solution, nous nous baserons sur une [`évaluation de la qualité d'image (EQI)`](https://www.researchgate.net/publication/322643565_Etude_comparative_des_methodes_d'evaluation_de_la_qualite_d'image). Il en existe trois catégorie:
- Les méthodes avec Référence: l'algorithme a un accès a une version parfaite de l'image avec laquelle il peut comparer la version déformée.
- Les méthodes sans Référence: l'algorithme n'a accès qu'à l'image déformée et doit estimer la qualité de l'image sans la connaissance de la version parfaite.
- Les méthodes avec Référence partielle: l'image de référence est disponible partiellement, elle est connu sous le nom d'estimation de la qualité de référence réduite

Il existe plusieur métrique d'EQI:
- L'erreur moyenne quadratique ou Mean Square Error (MSE) : comparer une image revalorisé avec l'image non dégrader, en se basant sur la MSE entre les pixels des deux images.
    $$MSE = \frac{1}{M*N} \sum_{m-1}^{M} \sum_{n-1}^{N}(I(m,n) - J(m,n)) $$ 
  $(M * N)$ est la taille de l'image, $I(m,n)$ et $J(m,n)$ sont les amplitudes des images parfaites et revalorisé  
  

In [1]:
def mse(imageA, imageB):
	# the 'Mean Squared Error' between the two images is the
	# sum of the squared difference between the two images;
	# NOTE: the two images must have the same dimension
	err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
	err /= float(imageA.shape[0] * imageA.shape[1])
	
	# return the MSE, the lower the error, the more "similar"
	# the two images are
	return err

- Le rapport crête signal sur bruit ou Peak Signal Noise Ratio (PSNR): au lieu de mesurer les distorsion, cette valeur mesure la fidélité, c'est à dire qu'elle est proportionnelle à la qualité.
    $$ PSNR = 10 \log_{10}(\frac{{l_{max}}^2}{MSE}) $$
    $l_{max}$ désigne la luminance maximale possible pour une image en niveau de gris. Si cette valeur est infinie cela veut dire que l'image est non dégardée, et plus la valeur est dégradée plus cette valeur décroît.  
Dans le cas des images couleur, aucune définition précisede l’adaptation de l’équation du PSNR n’existe. Plusieursvariantes existent :
   - Le PSNR est calculé sur chacun des trois plans colorimétriques puis une moyenne est ensuite effectuée afin d'obtenir une valeur finie
   - La distance euclidienne entre deux pixels de couleur est utilisée dans l'équation originale
cette mesure ne permet pas de prendre en compte la sensibilité du système visuelhumain, et n’apparaît pas être performante.

- Le rapport signal sur bruit ou Signal Noise Ratio (SNR): Variante du PSNR liant le MSE à l'énergie moyenne de l'image.
    $$ SNR = 10 \log_{10}(\frac{\frac{1}{N}\sum l^2}{MSE}) $$  


- L'index de Similarité Structurelle (SSIM): est une mesure de similarité entre deux images numériques plustôt que la différence pixel à pixel.  
    La  métrique  SSIM  est  calculée  sur  plusieurs  fenêtres d'une image de taille NxN. On dénote x et y  les vecteurs de l’image originale et déformée respectivement. 
    
    $$SSIM(x, y) = \frac{(2\mu_x\mu_y+c_1)(\sigma_{xy} + c_2)}{(\mu_{x}^{2} + \mu_{y}^{2} + c_1)(\sigma_{x}^{2} + \sigma_{y}^{2} + c_2)}$$  
    
    La méthode SSIM est clairement plus impliquée que la méthode MSE, mais l'essentiel est que la méthode SSIM tente de modéliser le changement perçu dans les informations structurelles de l'image.   
    Les paramètres de l'équation comprennent l'emplacement (x, y) de la fenêtre N x N dans chaque image, la moyenne des intensités($\sigma$) des pixels dans les directions x et y, la variance des intensités ($\mu$) dans les directions x et y, ainsi que la covariance ($c_1$ et $c_2$) qui sont des constantes. La valeur de SSIM peut varier entre -1 et 1, où 1 indique une similarité parfaite.

In [10]:
# import the necessary packages
from skimage.measure import compare_ssim as ssim
from skimage.measure import compare_mse as mse
from skimage.measure import compare_nrmse as nrmse
from skimage.measure import compare_psnr as psnr
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [12]:
def compare_images(imageA, imageB, title):
    # compute the mean squared error and structural similarity
    # index for the images
    m = mse(imageA, imageB)
    s = ssim(imageA, imageB, multichannel =True)
    n = nrmse(imageA, imageB)
    p = psnr(imageA, imageB)

    # setup the figure
    fig = plt.figure(title)
    plt.suptitle("MSE: %.2f, SSIM: %.2f, NRMSE: %.2f, PSNR: %.2f" % (m, s, n, p))

    # show first image
    ax = fig.add_subplot(1, 2, 1)
    plt.imshow(imageA)
    plt.axis("off")

    # show the second image
    ax = fig.add_subplot(1, 2, 2)
    plt.imshow(imageB)
    plt.axis("off")

    # show the images
    plt.show()

### Data Understanding :

A l'aide du dataSet fourni nous avons pu identifié 10 défauts sur les images:

* **Pinking** : Barre filtrante Magenta
* **Greening** : Barre filtrante Verte
* **Tearing** : Distortion, ligne déformante (I devient <)
* **Noise** : Bruis analogique (4 catégories types: gaussian - localvar - speckle - poisson)
* **Black Line** : Ligne noir fine horizontal
* **White Line** : Ligne blanche fine horizontal
* **Over-exposure** : Surexposition en luminausité
* **Blur**: traits des objets moins distincts
* **Color shifting** : Aberration chromatiques Rouges et Vertes
* **JPEG Artefact** : Artefactes de compression JPEG
* **Posterisation** : Réduction du nombre valeurs des dégradés

|Pinking|Greening|Tearing|Noise|Black Line|Posterisation|
|:-----:|:------:|:-----:|:---:|:--------:|:-----------:|
|![Pinking](../examples/pinking.png)|![Greening](../examples/greening.png)|![Tearing](../examples/tearing.png)|![Noise](../examples/noise.png)|![Black Line](../examples/black-line.png)|![Posterisation](../examples/posterisation.png)|
|**White Line**|**Over-exposure**|**Blur**|**Color Shifting**|**JPEG Artefact**|
|![White Line](../examples/white-line.png)|![Over-exposure](../examples/over-exposure.png)|![Blur](../examples/blur.png)|![Color Shifting](../examples/color-shifting.png)|![JPEG Artefact](../examples/jpeg-artefact.png)|

En observant la redondance des défauts nous avons défini une echelle de gravité de ces derniers sur les images:
1. Over-exposure  <!--add impact avec critère evaluation -->
2. Noise
3. Blur
4. Posterization
5. Greening/Pinking
6. Tearing
7. Color Shift
8. Black/White Line
9. JPEG Artefacts